# DSCI: 525 Milestone 1 - Group 8

## Rachel Wong, Rui Wang, Daniel Ortiz, Santiago Rugeles Schoonewolff

### Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

# Dask
import dask.dataframe as dd

# pyarrow and feather
import pyarrow.feather as feather
import pyarrow.dataset as ds

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

### Downloading the data

In [3]:
# Necessary metadata
article_id = 14096681  # unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

### Unzipping the data

In [5]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 2.74 s, sys: 2.34 s, total: 5.08 s
Wall time: 1min 17s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 18.6 s, sys: 3.38 s, total: 22 s
Wall time: 23 s


### Combining data CSVs using Pandas

In [7]:
df = pd.read_csv("./figshare/ACCESS-CM2_daily_rainfall_NSW.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: './figshare/ACCESS-CM2_daily_rainfall_NSW.csv'

In [ ]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together
## here we are using a normal python way of merging the data 

files = glob.glob('figshare/*.csv') # load all the CSVs
df = pd.concat((pd.read_csv(file, index_col=0) # combine them all
                .assign(model=re.findall(r'/([^_]*)', file)[0])
                for file in files)
              )
df.to_csv("figshare/combined_data.csv")

In [ ]:
df_combined = pd.read_csv("./figshare/combined_data.csv")
df_combined # combined dataframe 

discussion about the different computer times on each of our machines

In [ ]:
df_combined["model"].unique() # print out the unique models

In [ ]:
%%sh
du -sh figshareairline/combined_data.csv

We can see from our combined dataframe that we have 29 unique models to continue our analysis with.

### Load the combined CSV to memory and perform a simple EDA

In [ ]:
%%time
%%memit
#simple pandas - This is how we do normally ,which means we are loading the entire data to the memory
df = pd.read_csv("figshare/combined_data.csv")
print(df["model"].value_counts())

In [ ]:
df.head()

In [ ]:
# checking datatypes for columns
df.dtypes

We can see that we have object and float64 type columns in our dataframe. This makes sense that `time` and `model` are object types and the rest such as latitude, longitude, and rain are float64 types.

### Investigate changing the `dtype` of our data

In [ ]:
print(f"The memory usage with the original float64 dtype: {df[['lat_min','lat_max','rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"The memory usage after changing to float32 dtype: {df[['lat_min','lat_max','rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

discussion about reducing memory usage of our numeric data types by switching from float64 to float32

### Loading our data using Dask and checking the value counts of models

In [ ]:
%%time
%%memit
# Dask
df_dask = dd.read_csv('figshare/combined_data.csv')
print(df_dask["model"].value_counts().compute())

discussion about reducing memory usage with Dask

### Transfering the dataframe from Python to R using Feather

In [ ]:
%%time
%%memit
dataset = ds.dataset("figshare/combined_data.csv", format="csv")
## this is of arrow table format
table = dataset.to_table()

In [ ]:
%%time
# writing in feather format
feather.write_feather(table, 'figshare/combined_data.feather')

discussion of memory and time using feather, and why we chose feather

### Simple EDA in R

In [ ]:
%%R
library(tidyr)

In [ ]:
%%time
%%R
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figshare/combined_data.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model) # showing the different counts of the models 
end_time <- Sys.time()
print(result)
print(end_time - start_time)

In [ ]:
%%R
result <- r_table %>% count(time) # showing the different counts of the time
print(result)

discussion about the different counts using R, time and memory differences

In [ ]:
%%R
r_table_d <- r_table %>% drop_na() # drop NA values

In [ ]:
%%R
r_table_d <- r_table_d %>% rename(rain_mmperday = `rain (mm/day)`) # rename the column for rain

In [ ]:
%%R
Columns <- c("lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/perday)")
Mean <- c(mean(r_table_d$lat_min), mean(r_table_d$lat_max), mean(r_table_d$lon_min), mean(r_table_d$lon_max), mean(r_table_d$rain_mmperday))
Mode <- c(mode(r_table_d$lat_min), mode(r_table_d$lat_max), mode(r_table_d$lon_min), mode(r_table_d$lon_max), mode(r_table_d$rain_mmperday))
Median <- c(median(r_table_d$lat_min), median(r_table_d$lat_max), median(r_table_d$lon_min), median(r_table_d$lon_max), median(r_table_d$rain_mmperday))

result <- data.frame(Columns, Mean, Mode, Median)
print(result)

discussion about the different mean, median, modes of the columns analyzed using R